# Previsione delle vendite con variabili esogene

## Contesto

L'obiettivo di questo notebook è mostrare, in modo semplice e didattico, come si possa prevedere la domanda giornaliera di un prodotto utilizzando anche **variabili esterne** (dette "esogene"), come il prezzo, le promozioni o il meteo.

Simuleremo dei dati e useremo tre modelli di previsione, confrontandone i risultati. I passaggi sono gli stessi che si seguirebbero in un progetto reale di previsione della domanda.

In [ ]:
import datetime

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_absolute_error, root_mean_squared_error
from sklearn.model_selection import GridSearchCV, TimeSeriesSplit
from sklearn.preprocessing import StandardScaler
from statsmodels.tsa.seasonal import seasonal_decompose
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from xgboost import XGBRegressor

## Simulazione del dataset

In [ ]:
np.random.seed(42)
n_days = 365 * 2
date_range = pd.date_range(start=datetime.datetime.now(), periods=n_days, freq="D")

temperature = (
    10
    + 15 * np.sin(2 * np.pi * date_range.dayofyear / 365)
    + np.random.normal(0, 2, n_days)
)
price = np.random.uniform(8, 12, n_days)
promotion = np.random.binomial(1, 0.3, n_days)

df = pd.DataFrame(
    {
        "temperature": temperature,
        "price": price,
        "promotion": promotion,
    },
    index=date_range,
)

target = (
    50
    - 2 * price
    + 20 * promotion
    + 0.5 * temperature
    - 0.05 * temperature**2
    + 10 * np.sin(2 * np.pi * date_range.dayofyear / 365)
    + 5 * np.cos(2 * np.pi * date_range.dayofyear / (365 / 2))
    + np.random.normal(0, 3, n_days)
)

df["sales"] = pd.Series(target, index=date_range, name="sales")

### Che cosa contiene il nostro dataset?

Ogni riga del dataset rappresenta un giorno. Per ogni giorno abbiamo:

- `sales`: il numero di vendite giornaliere (da prevedere)
- `price`: il prezzo del prodotto in quel giorno
- `promotion`: una variabile che vale 1 se il prodotto è in promozione, 0 altrimenti
- `temperature`: la temperatura del giorno (es. può influire sulle vendite stagionali)

Abbiamo anche introdotto un po' di **rumore casuale**, per simulare l'imprevedibilità reale del mercato.

## Analisi esplorativa

In [ ]:
df[["sales", "price", "promotion", "temperature"]].plot(subplots=True, figsize=(12, 8))
plt.tight_layout()
plt.show()

### Perché osserviamo i dati?

Guardare i dati nel tempo ci permette di:

- Capire se ci sono stagionalità settimanali o annuali
- Vedere l’effetto di promozioni e temperature
- Identificare anomalie o trend

Questa fase visiva è fondamentale per guidare le decisioni successive sui modelli da usare.

## Analisi della Stagionalità

La stagionalità rappresenta pattern ricorrenti e prevedibili nei dati nel tempo, come variazioni mensili o trimestrali nelle vendite dovute a fattori stagionali (es. festività, stagioni climatiche, abitudini di consumo).

Indagare la stagionalità ci permette di migliorare la qualità delle previsioni, introducendo variabili che aiutano i modelli a catturare questi effetti ricorrenti.

In questa sezione:
- Visualizzeremo la stagionalità tramite decomposizione della serie.
- Creeremo variabili temporali per includere la stagionalità nei modelli di previsione.

In [ ]:
result = seasonal_decompose(df["sales"], model="additive", period=12)

plt.figure(figsize=(12, 8))
plt.subplot(411)
plt.plot(df["sales"], label="Originale")
plt.legend(loc="upper left")
plt.subplot(412)
plt.plot(result.trend, label="Trend")
plt.legend(loc="upper left")
plt.subplot(413)
plt.plot(result.seasonal, label="Stagionalità")
plt.legend(loc="upper left")
plt.subplot(414)
plt.plot(result.resid, label="Residui")
plt.legend(loc="upper left")
plt.tight_layout()
plt.show()

## Feature Engineering

Aggiungiamo variabili stagionali, medie mobili e lag per catturare la dinamica temporale.

In [ ]:
df["dayofweek"] = df.index.dayofweek
df["month"] = df.index.month
df = pd.get_dummies(df, columns=["dayofweek", "month"], drop_first=True)

df["sales_ma7"] = df["sales"].rolling(window=7).mean()
df["sales_lag1"] = df["sales"].shift(1)

df.dropna(inplace=True)

### Cosa sono le "feature" e perché le costruiamo?

Le **feature** (o variabili esplicative) sono gli elementi che useremo per prevedere le vendite. Oltre a quelle già presenti nel dataset (prezzo, promozione...), aggiungiamo:

- Giorno della settimana (`dayofweek`) e mese (`month`) per catturare la stagionalità
- Media mobile a 7 giorni (`sales_ma7`): serve a catturare la tendenza recente
- Valore del giorno precedente (`sales_lag1`): perché le vendite di oggi sono spesso simili a quelle di ieri

Queste trasformazioni aiutano i modelli a cogliere meglio i comportamenti ricorrenti.

## Divisione tra dati di addestramento e di test

In ogni progetto di previsione è fondamentale **valutare le prestazioni del modello su dati "mai visti" prima**, cioè non usati per costruirlo. Per questo, dividiamo i dati in due parti:

- **Training set**: i primi 80% dei giorni. Servono per far "imparare" il modello.
- **Test set**: gli ultimi 20% dei giorni. Servono per verificare se il modello sa generalizzare su dati nuovi.

Poiché stiamo lavorando con dati **temporali**, è molto importante **rispettare l’ordine cronologico**: non possiamo mescolare i giorni casualmente, altrimenti finiremmo per "barare", usando informazioni dal futuro per prevedere il passato.

In [ ]:
# Calcoliamo l'indice di separazione: 80% per il training, 20% per il test
split_index = int(len(df) * 0.8)

# Usiamo solo le variabili rilevanti per la previsione
features = ["price", "promotion", "temperature"]
target = "sales"

# Dividiamo i dati mantenendo l'ordine temporale
X_train = df[features].iloc[:split_index]
X_test = df[features].iloc[split_index:]

y_train = df[target].iloc[:split_index]
y_test = df[target].iloc[split_index:]

## Perché normalizziamo i dati?

Molti modelli di machine learning (come la regressione lineare e le reti neurali) funzionano meglio se tutte le variabili numeriche sono **sullo stesso ordine di grandezza**.

Ad esempio, il prezzo potrebbe variare tra 5 e 10, mentre le vendite vanno da 50 a 200. Questo può sbilanciare il modello.

Per questo, applichiamo una **normalizzazione** (scaling) che trasforma ogni variabile in modo che abbia:

- media = 0  
- deviazione standard = 1

È importante eseguire questo processo **solo sui dati di addestramento**, e poi applicarlo ai dati di test, per non introdurre informazioni future nella fase di training.

In [ ]:
# Creiamo uno scaler e lo "addestriamo" solo sui dati di training
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)

# Applichiamo la stessa trasformazione ai dati di test
X_test_scaled = scaler.transform(X_test)

## Cosa sono i modelli di regressione?

Un **modello di regressione** cerca di trovare una relazione matematica tra le feature (es. prezzo, promozione...) e il valore che vogliamo prevedere (vendite).

Cominceremo con un modello molto semplice: la **Ridge Regression**, che è una variante della regressione lineare. È utile come punto di partenza perché è veloce, interpretabile e spesso sorprendentemente efficace.

In [ ]:
ridge = Ridge()
param_grid = {"alpha": [0.1, 1.0, 10.0, 100.0]}
tscv = TimeSeriesSplit(n_splits=5)
grid = GridSearchCV(ridge, param_grid, cv=tscv)
grid.fit(X_train_scaled, y_train)

y_pred_ridge = grid.predict(X_test_scaled)

## Cos’è XGBoost?

**XGBoost** è un modello più sofisticato. Si basa su una tecnica chiamata *alberi decisionali a gradient boosting*. Funziona bene anche in situazioni non lineari e con molte interazioni tra le variabili.

Rispetto alla regressione lineare, può cogliere relazioni più complesse.

In [ ]:
xgb = XGBRegressor(n_estimators=100, learning_rate=0.1)
xgb.fit(X_train, y_train)
y_pred_xgb = xgb.predict(X_test)

## Cosa sono le reti neurali?

Le **reti neurali** sono modelli ispirati al cervello umano. Sono molto flessibili e capaci di adattarsi a una vasta gamma di problemi. Qui useremo una rete semplice (2 layer nascosti) per vedere come si comporta rispetto ai modelli precedenti.

Tuttavia, sono meno interpretabili e richiedono un po' più di dati per dare il meglio.

In [ ]:
model = Sequential(
    [
        Input(shape=(X_train_scaled.shape[1],)),
        Dense(32, activation="relu"),
        Dense(16, activation="relu"),
        Dense(1),
    ]
)
model.compile(optimizer=Adam(learning_rate=0.01), loss="mse")

es = EarlyStopping(patience=10, restore_best_weights=True)
model.fit(
    X_train_scaled, y_train, validation_split=0.2, epochs=100, callbacks=[es], verbose=0
)

y_pred_mlp = model.predict(X_test_scaled).flatten()

## Valutazione dei modelli

In [ ]:
def evaluate(y_true, y_pred, name):
    mae = mean_absolute_error(y_true, y_pred)
    rmse = root_mean_squared_error(y_true, y_pred)
    print(f"{name} → MAE: {mae:.2f}, RMSE: {rmse:.2f}")


evaluate(y_test, y_pred_ridge, "Ridge")
evaluate(y_test, y_pred_xgb, "XGBoost")
evaluate(y_test, y_pred_mlp, "MLP")

## Come si valuta un modello?

Per confrontare i modelli usiamo due indicatori:

- **MAE**: Errore Assoluto Medio, misura quanto in media sbaglia il modello
- **RMSE**: Scarto Quadratico Medio, penalizza maggiormente gli errori grandi

Più questi valori sono bassi, migliore è la previsione.

## Confronto delle predizioni

In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(y_test.index, y_test, label="True")
plt.plot(y_test.index, y_pred_ridge, label="Ridge")
plt.plot(y_test.index, y_pred_xgb, label="XGBoost")
plt.plot(y_test.index, y_pred_mlp, label="MLP")
plt.legend()
plt.title("Confronto delle predizioni")
plt.show()

### Cosa osserviamo nel confronto?

Guardando il grafico, possiamo confrontare:

- L’andamento reale delle vendite (**True**)
- Le previsioni dei diversi modelli

Questo ci aiuta a capire visivamente quale modello segue meglio la realtà e se ci sono differenze significative tra le soluzioni.

In [ ]:
ape_ridge = np.abs((y_test - y_pred_ridge) / y_test) * 100
ape_xgb = np.abs((y_test - y_pred_xgb) / y_test) * 100
ape_mlp = np.abs((y_test - y_pred_mlp) / y_test) * 100

fig, axes = plt.subplots(1, 3, figsize=(18, 5), sharey=True)

axes[0].hist(ape_ridge, bins=30, color="tab:blue", alpha=0.7)
axes[0].set_title("Ridge")
axes[0].set_xlabel("Errore Percentuale Assoluto (%)")
axes[0].set_ylabel("Frequenza")

axes[1].hist(ape_xgb, bins=30, color="tab:orange", alpha=0.7)
axes[1].set_title("XGBoost")
axes[1].set_xlabel("Errore Percentuale Assoluto (%)")

axes[2].hist(ape_mlp, bins=30, color="tab:green", alpha=0.7)
axes[2].set_title("MLP")
axes[2].set_xlabel("Errore Percentuale Assoluto (%)")

plt.suptitle("Distribuzione degli errori percentuali assoluti per modello")
plt.tight_layout(rect=[0, 0, 1, 0.95])
plt.show()

### Distribuzione degli errori percentuali assoluti per modello

Il grafico mostra tre istogrammi affiancati, uno per ciascun modello di previsione (Ridge, XGBoost e MLP). Ogni istogramma rappresenta la distribuzione dell'errore percentuale assoluto (APE) calcolato per ogni osservazione nel test set.

L’errore percentuale assoluto misura la deviazione relativa tra il valore predetto e quello reale, espressa in percentuale. Ad esempio, un valore del 5% indica che la previsione si discosta dal dato reale del 5%.

Questi istogrammi permettono di confrontare in modo visivo:
- La precisione complessiva di ogni modello (quanto sono concentrati gli errori vicino allo zero).
- La presenza di errori molto grandi (code dell’istogramma verso valori alti).
- La consistenza della previsione nel tempo.

In sintesi, modelli con una distribuzione degli errori più stretta e concentrata verso valori bassi sono da preferire, perché forniscono previsioni più accurate e stabili.

## Conclusioni

In questo notebook abbiamo implementato e confrontato tre diversi modelli per la previsione delle vendite con variabili esogene: Ridge Regression, XGBoost e una rete neurale MLP.

Dal confronto emergono alcune evidenze importanti:

- **Importanza della normalizzazione e dello split temporale:** Il corretto preprocessing dei dati è essenziale per evitare informazioni spurie e garantire una stima realistica delle performance future.

- **Analisi della stagionalità:** L’inserimento di variabili che rappresentano la stagionalità migliora la capacità predittiva dei modelli, evidenziando come la componente temporale ciclica sia fondamentale nella previsione delle vendite.

- **Analisi degli errori:** L’istogramma degli errori percentuali assoluti permette di valutare non solo l’accuratezza media, ma anche la distribuzione e la frequenza degli errori maggiori, fornendo una visione più completa della qualità delle previsioni.

In sintesi, la combinazione di tecniche di machine learning con un’accurata fase di esplorazione e preprocessing dei dati rappresenta la strategia migliore per affrontare problemi di forecasting complessi come quello presentato. Per applicazioni reali, è consigliabile testare ulteriori modelli e iperparametri, nonché monitorare costantemente le performance con dati aggiornati.
